In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
os.chdir("/Users/seungji/Desktop/Dacon/Daesamanlap")

**행정동 법정동 매핑**
- 출처: 행정안전부
- 링크: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do;jsessionid=+ZUzK-vRbyaKqumcJq8NrYJf.node50?bbsId=BBSMSTR_000000000052&nttId=85215

In [3]:
mapping = pd.read_excel("External2/법정동행정동매핑.xlsx")
mapping

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,nan
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,nan
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,nan
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,nan
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,nan
...,...,...,...,...,...,...,...,...
21734,5013061000,제주특별자치도,서귀포시,중문동,5013011400,대포동,20060701,nan
21735,5013061000,제주특별자치도,서귀포시,중문동,5013011800,하원동,20060701,nan
21736,5013062000,제주특별자치도,서귀포시,예래동,5013011900,색달동,20060701,nan
21737,5013062000,제주특별자치도,서귀포시,예래동,5013012000,상예동,20060701,nan


#### 건축연면적

In [4]:
bld_tr = pd.read_csv("ProcessedData/building_tr.csv")
bld_tst = pd.read_csv("ProcessedData/building_tst.csv")

#### 법정동별 인구수 및 세대수 및 ... 

- 링크: https://kostat.go.kr/portal/korea/index.action

In [5]:
pop1 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106주민등록인구및세대현황_월간.xlsx")
pop2 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106연령별인구현황_월간.xlsx")

- 건축연면적의 지번주소는 법정동 기준으로 되어있다. 
- 법정동별 인구정보는 행정동 기준으로 되어있다. 
- 이 두 데이터를 잇기 위해 행정동/법정동 매핑 데이터가 필요하다. 

#### 순서
1. `mapping`에 `pop1`과 `pop2`를 merge -> `mapping1`
    - key: mapping의 `행정동코드`와 pop1과 pop2의 `행정기관코드`를 기준으로
    - 
1. `건축연면적`에 `mapping1`를 merge
    - key: 

## 1. mapping에 pop1과 pop2를 merge -> mapping2

In [6]:
len(set(mapping.행정동코드)& set(pop1.행정기관코드))

3835

In [7]:
len(set(mapping.행정동코드)& set(pop2.행정기관코드))

3837

In [8]:
pop1[pop1.행정기관코드.isin(list(set(pop2.행정기관코드)-set(mapping.행정동코드)))]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
434,1174052000,서울특별시 강동구 상일동,38683,13380,2.8900,19080,19603,0.9700
833,2811060000,인천광역시 중구 북성동,2744,1507,1.8200,1423,1321,1.0800
834,2811061000,인천광역시 중구 송월동,4378,2153,2.0300,2157,2221,0.9700
1615,4139059500,경기도 시흥시 배곧동,71154,27314,2.6100,36445,34709,1.0500
1704,4148038700,경기도 파주시 장단출장소,552,216,2.5600,278,274,1.0100
1705,4148040000,경기도 파주시 진동면,164,74,2.2200,90,74,1.2200
1789,4159400000,경기도 화성시동탄출장소,0,0,0.0000,0,0,0.0000
3209,4719052000,경상북도 구미시 원평1동,3716,2711,1.3700,2074,1642,1.2600
3210,4719053000,경상북도 구미시 원평2동,2757,1668,1.6500,1443,1314,1.1000
3473,4812155000,경상남도 창원시 의창구 용지동,29735,13242,2.2500,14933,14802,1.0100


In [9]:
{"서울특별시 강동구 상일동":1174052500,}

{'서울특별시 강동구 상일동': 1174052500}

In [10]:
pop1.행정기관코드.value_counts().value_counts()


1    3846
Name: 행정기관코드, dtype: int64

In [11]:
pop2.행정기관코드.value_counts().value_counts()


1    3847
Name: 행정기관코드, dtype: int64

In [12]:
print(mapping.shape)

(21739, 8)


In [13]:
mapping1 = mapping.merge(pop1,left_on = "행정동코드",right_on = "행정기관코드",how = "left")

In [14]:
print(mapping1.shape)

(21739, 16)


In [15]:
mapping1 = mapping1.merge(pop2,left_on = "행정동코드",right_on = "행정기관코드",how = "left")

In [16]:
print(mapping1.shape)

(21739, 57)


In [17]:
print(sum(mapping1.행정기관코드_x != mapping1.행정기관코드_y))
print(sum(mapping1.총인구수_x != mapping1.총인구수_y))
print(sum(mapping1.행정기관_x != mapping1.행정기관_y))

84
84
84


In [18]:
mapping1[mapping1.행정기관코드_x != mapping1.행정기관코드_y]

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자,행정기관코드_x,행정기관_x,총인구수_x,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,행정기관코드_y,행정기관_y,총인구수_y,총연령구간인구수,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
751,1174052500,서울특별시,강동구,상일제1동,1174010300,상일동,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
752,1174052600,서울특별시,강동구,상일제2동,1174010300,상일동,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1589,2811061500,인천광역시,중구,개항동,2811013700,전동,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1590,2811061500,인천광역시,중구,개항동,2811013800,북성동1가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1591,2811061500,인천광역시,중구,개항동,2811013900,북성동2가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1592,2811061500,인천광역시,중구,개항동,2811014000,북성동3가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1593,2811061500,인천광역시,중구,개항동,2811014100,선린동,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1594,2811061500,인천광역시,중구,개항동,2811014200,송월동1가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1595,2811061500,인천광역시,중구,개항동,2811014300,송월동2가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1596,2811061500,인천광역시,중구,개항동,2811014400,송월동3가,20210701,nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## 2. 건축연면적에 mapping2를 merge


In [19]:
bld_tr.head()

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수,가구수,도로명주소,지번주소
0,47113-100216867,장량휴먼시아,0.0000,632.9550,"8,409.5540","8,394.6490",0.0000,149.0000,0.0000,경상북도 포항시 북구 장량중앙로 17,경상북도 포항시 북구 양덕동 336-1
1,48220-100186682,미수휴먼시아,0.0000,661.9600,"1,213.8700",240.8300,0.0000,0.0000,0.0000,경상남도 통영시 미우지해안로 107,경상남도 통영시 미수동 695
2,30200-100200744,도안휴먼시아4단지,0.0000,707.1500,"4,636.4100","4,636.4100",0.0000,102.0000,0.0000,대전광역시 유성구 상대로 40,대전광역시 유성구 상대동 465
3,41360-100216608,휴먼시아,0.0000,385.9900,"6,017.7600","5,951.0800",0.0000,76.0000,0.0000,경기도 남양주시 늘을3로 8,경기도 남양주시 호평동 712
4,45140-100190554,장신휴먼시아1단지,0.0000,556.1800,"6,611.9000","6,611.9000",0.0000,84.0000,0.0000,전라북도 익산시 오산면 선화로4길 66,전라북도 익산시 오산면 장신리 1130


In [20]:
len(bld_tr)

421

In [26]:
juso_tr = list(map(lambda x: x.split()[:-1] , bld_tr.지번주소))
juso_tr2 = list(map(lambda x: x.split()[:2] , bld_tr.지번주소))
#for idx, j in enumerate (bld_tr.지번주소):
    #jj = j.split()[:-1]
 #   if j.split()[:-1][-1][-1] == "리":
  #      juso_tr[idx] = j.split()[:-1][:-1]
juso_tr = list(map(lambda x: ''.join(x) , juso_tr))
juso_tr2 = list(map(lambda x: ''.join(x) , juso_tr2))
juso_tr

['경상북도포항시북구양덕동',
 '경상남도통영시미수동',
 '대전광역시유성구상대동',
 '경기도남양주시호평동',
 '전라북도익산시오산면장신리',
 '전라북도익산시오산면장신리',
 '경기도성남시분당구백현동',
 '대전광역시유성구지족동',
 '강원도횡성군횡성읍읍하리',
 '광주광역시남구백운동',
 '광주광역시남구백운동',
 '강원도강릉시교동',
 '강원도강릉시포남동',
 '강원도강릉시입암동',
 '경상남도거제시장평동',
 '충청남도당진시석문면통정리',
 '충청남도공주시옥룡동',
 '부산광역시기장군기장읍교리',
 '경상남도김해시구산동',
 '경상남도김해시삼문동',
 '대전광역시서구관저동',
 '대전광역시서구관저동',
 '대전광역시서구월평동',
 '대전광역시중구문화동',
 '대전광역시유성구반석동',
 '대전광역시대덕구법동',
 '대전광역시동구낭월동',
 '대전광역시중구중촌동',
 '대전광역시동구천동',
 '대전광역시동구판암동',
 '대전광역시동구판암동',
 '강원도동해시천곡동',
 '경상남도창원시마산회원구내서읍삼계리',
 '경상남도창원시마산회원구내서읍상곡리',
 '경상남도밀양시가곡동',
 '경상남도밀양시삼문동',
 '충청남도보령시동대동',
 '충청남도보령시명천동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시영도구동삼동',
 '부산광역시영도구동삼동',
 '부산광역시북구덕천동',
 '부산광역시북구만덕동',
 '부산광역시사상구모라동',
 '부산광역시해운대구반송동',
 '부산광역시북구금곡동',
 '충청남도부여군부여읍쌍북리',
 '경상남도사천시벌리동',
 '제주특별자치도서귀포시동홍동',
 '충청남도서산시석림동',
 '강원도속초시교동',
 '울산광역시울주군삼남읍교동리',
 '울산광역시북구화봉동',
 '강원도원주시문막읍건등리',
 '부산광역시기장군정관읍용수리',
 '제주특별자치도제주시아라일동',
 '제주특별자치도제주시노형동',
 '제주특별자치도제주시화북일동',
 '경상남도진주시가좌동',
 '경상남도진주

In [22]:
bld_tr["juso"] = juso_tr

In [27]:
juso_tst = list(map(lambda x: x.split()[:-1] , bld_tst.지번주소))
juso_tst2 = list(map(lambda x: x.split()[:2] , bld_tst.지번주소))
#for idx, j in enumerate (bld_tst.지번주소):
    #jj = j.split()[:-1]
 #   if j.split()[:-1][-1][-1] == "리":
  #      juso_tst[idx] = j.split()[:-1][:-1]
juso_tst = list(map(lambda x: ''.join(x) , juso_tst))
juso_tst2 = list(map(lambda x: ''.join(x) , juso_tst2))
juso_tst

['경기도광명시소하동',
 '경기도광명시소하동',
 '부산광역시동래구안락동',
 '전라북도익산시오산면장신리',
 '경기도성남시분당구삼평동',
 '경기도성남시분당구백현동',
 '전라북도군산시미장동',
 '전라북도김제시교동',
 '경상남도거제시일운면소동리',
 '경상남도남해군남해읍평리',
 '충청남도논산시대교동',
 '대전광역시서구월평동',
 '대전광역시동구삼성동',
 '부산광역시부산진구당감동',
 '부산광역시사상구모라동',
 '부산광역시해운대구좌동',
 '제주특별자치도서귀포시동홍동',
 '강원도속초시조양동',
 '충청남도아산시읍내동',
 '경상남도양산시상북면대석리',
 '울산광역시남구달동',
 '울산광역시북구명촌동',
 '울산광역시북구호계동',
 '울산광역시북구호계동',
 '울산광역시북구화봉동',
 '울산광역시동구화정동',
 '강원도원주시명륜동',
 '강원도원주시무실동',
 '강원도원주시무실동',
 '강원도원주시태장동',
 '부산광역시기장군정관읍모전리',
 '제주특별자치도제주시애월읍고성리',
 '제주특별자치도제주시노형동',
 '충청남도천안시서북구쌍용동',
 '강원도춘천시효자동',
 '경상북도의성군의성읍상리리',
 '충청북도제천시강제동',
 '경기도용인시기흥구서천동',
 '경기도용인시기흥구농서동',
 '충청북도음성군삼성면덕정리',
 '전라북도익산시함열읍와리',
 '경상북도경산시상방동',
 '경상북도고령군다산면상곡리',
 '경기도광명시소하동',
 '경기도광명시소하동',
 '광주광역시서구동천동',
 '광주광역시남구노대동',
 '광주광역시광산구흑석동',
 '광주광역시북구양산동',
 '경상북도구미시구평동',
 '경기도군포시부곡동',
 '전라북도김제시검산동',
 '경기도남양주시진접읍금곡리',
 '대전광역시유성구하기동',
 '충청남도논산시취암동',
 '전라남도담양군담양읍백동리',
 '대구광역시달성군다사읍매곡리',
 '대전광역시동구가오동',
 '경기도동두천시생연동',
 '강원도동해시송정동',
 '전라남도무안군삼향읍남악리',
 '경기도부천시소사본동',
 '경기도부천시여월동',


In [34]:
juso_tst2

['경기도광명시',
 '경기도광명시',
 '부산광역시동래구',
 '전라북도익산시',
 '경기도성남시',
 '경기도성남시',
 '전라북도군산시',
 '전라북도김제시',
 '경상남도거제시',
 '경상남도남해군',
 '충청남도논산시',
 '대전광역시서구',
 '대전광역시동구',
 '부산광역시부산진구',
 '부산광역시사상구',
 '부산광역시해운대구',
 '제주특별자치도서귀포시',
 '강원도속초시',
 '충청남도아산시',
 '경상남도양산시',
 '울산광역시남구',
 '울산광역시북구',
 '울산광역시북구',
 '울산광역시북구',
 '울산광역시북구',
 '울산광역시동구',
 '강원도원주시',
 '강원도원주시',
 '강원도원주시',
 '강원도원주시',
 '부산광역시기장군',
 '제주특별자치도제주시',
 '제주특별자치도제주시',
 '충청남도천안시',
 '강원도춘천시',
 '경상북도의성군',
 '충청북도제천시',
 '경기도용인시',
 '경기도용인시',
 '충청북도음성군',
 '전라북도익산시',
 '경상북도경산시',
 '경상북도고령군',
 '경기도광명시',
 '경기도광명시',
 '광주광역시서구',
 '광주광역시남구',
 '광주광역시광산구',
 '광주광역시북구',
 '경상북도구미시',
 '경기도군포시',
 '전라북도김제시',
 '경기도남양주시',
 '대전광역시유성구',
 '충청남도논산시',
 '전라남도담양군',
 '대구광역시달성군',
 '대전광역시동구',
 '경기도동두천시',
 '강원도동해시',
 '전라남도무안군',
 '경기도부천시',
 '경기도부천시',
 '강원도삼척시',
 '강원도삼척시',
 '제주특별자치도서귀포시',
 '부산광역시해운대구',
 '경상남도양산시',
 '경기도양주시',
 '경기도양주시',
 '경기도양주시',
 '제주특별자치도제주시',
 '경기도용인시',
 '경기도용인시',
 '경기도용인시',
 '경기도용인시',
 '경기도용인시',
 '경기도용인시',
 '울산광역시울주군',
 '대구광역시동구',
 '대구광역시동구',
 '경상남도김해시',
 '충청북도음성군',
 '경

In [28]:
bld_tst["juso"] = juso_tst

In [29]:
mapping1.시군구명 = mapping1.시군구명.fillna('')
mapping1.읍면동명 = mapping1.읍면동명.fillna('')
mapping1.동리명 = mapping1.동리명.fillna('')
mapping1.행정기관_x = mapping1.행정기관_x.fillna('')

In [30]:
temp = []
for i in range(len(mapping1)):
        
    if mapping1.읍면동명[i]!= '':
        if mapping1.읍면동명[i].strip()[-1] in ["면","읍"]:
            temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["읍면동명"][i].strip()+mapping1["동리명"][i].strip())
        else:
            temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["동리명"][i].strip())
    else: 
        temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["동리명"][i].strip())
     
    
mapping1["주소"] = temp

In [32]:
len(mapping1), len(juso_)

21739

In [237]:
mapping1.행정기관_x = list(map(lambda x: x.replace(" ",'') , mapping1.행정기관_x))
mapping1.주소 = list(map(lambda x: x.replace(" ",'') , mapping1.주소))

In [238]:
len(set(bld_tr.juso) - set(mapping1.주소))

0

In [239]:
set(bld_tr.juso) - set(mapping1.주소)

set()

In [240]:
len(set(bld_tst.juso) - set(mapping1.주소))

0

In [241]:
mapping2 = mapping1.groupby("주소").mean().reset_index()

In [242]:
mrg_tr = bld_tr.merge(mapping2,left_on = "juso",right_on = "주소",how = "left")
mrg_tr

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수_x,가구수,도로명주소,지번주소,juso,주소,행정동코드,법정동코드,생성일자,말소일자,행정기관코드_x,총인구수_x,세대수_y,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
0,47113-100216867,장량휴먼시아,0.0000,632.9550,"8,409.5540","8,394.6490",0.0000,149.0000,0.0000,경상북도 포항시 북구 장량중앙로 17,경상북도 포항시 북구 양덕동 336-1,경상북도포항시북구양덕동,경상북도포항시북구양덕동,"4,711,370,000.0000","4,711,312,200.0000","19,950,101.0000",nan,"4,711,370,000.0000","73,070.0000","29,115.0000",...,"6,528.0000","5,974.0000","4,133.0000","1,247.0000",269.0000,20.0000,0.0000,"36,038.0000","36,038.0000","3,847.0000","4,215.0000","4,084.0000","5,001.0000","6,892.0000","6,151.0000","3,898.0000","1,219.0000",611.0000,117.0000,3.0000
1,48220-100186682,미수휴먼시아,0.0000,661.9600,"1,213.8700",240.8300,0.0000,0.0000,0.0000,경상남도 통영시 미우지해안로 107,경상남도 통영시 미수동 695,경상남도통영시미수동,경상남도통영시미수동,"4,822,066,500.0000","4,822,011,500.0000","20,101,231.0000",nan,"4,822,066,500.0000","11,129.0000","4,832.0000",...,967.0000,980.0000,935.0000,332.0000,64.0000,5.0000,0.0000,"5,662.0000","5,662.0000",396.0000,645.0000,457.0000,554.0000,942.0000,"1,032.0000",955.0000,435.0000,216.0000,29.0000,1.0000
2,30200-100200744,도안휴먼시아4단지,0.0000,707.1500,"4,636.4100","4,636.4100",0.0000,102.0000,0.0000,대전광역시 유성구 상대로 40,대전광역시 유성구 상대동 465,대전광역시유성구상대동,대전광역시유성구상대동,"3,020,061,000.0000","3,020,011,500.0000","20,130,916.0000",nan,"3,020,061,000.0000","45,840.0000","18,979.0000",...,"4,132.0000","3,110.0000","2,203.0000",757.0000,221.0000,16.0000,0.0000,"23,897.0000","23,897.0000","2,584.0000","2,495.0000","2,547.0000","4,013.0000","4,584.0000","3,623.0000","2,630.0000",864.0000,440.0000,113.0000,4.0000
3,41360-100216608,휴먼시아,0.0000,385.9900,"6,017.7600","5,951.0800",0.0000,76.0000,0.0000,경기도 남양주시 늘을3로 8,경기도 남양주시 호평동 712,경기도남양주시호평동,경기도남양주시호평동,"4,136,051,000.0000","4,136,010,100.0000","19,950,101.0000",nan,"4,136,051,000.0000","57,499.0000","21,736.0000",...,"5,771.0000","4,256.0000","2,404.0000","1,353.0000",474.0000,45.0000,2.0000,"29,479.0000","29,479.0000","3,039.0000","3,501.0000","3,001.0000","3,994.0000","6,191.0000","4,149.0000","2,917.0000","1,778.0000",750.0000,150.0000,9.0000
4,45140-100190554,장신휴먼시아1단지,0.0000,556.1800,"6,611.9000","6,611.9000",0.0000,84.0000,0.0000,전라북도 익산시 오산면 선화로4길 66,전라북도 익산시 오산면 장신리 1130,전라북도익산시오산면장신리,전라북도익산시오산면장신리,"4,514,031,000.0000","4,514,031,022.0000","19,950,510.0000",nan,"4,514,031,000.0000","7,783.0000","3,889.0000",...,545.0000,676.0000,733.0000,391.0000,173.0000,19.0000,2.0000,"3,931.0000","3,931.0000",254.0000,304.0000,307.0000,333.0000,478.0000,634.0000,750.0000,479.0000,324.0000,68.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,50130-100340536,서귀포서홍 행복주택,0.0000,0.0000,67.9800,0.0000,0.0000,0.0000,0.0000,제주특별자치도 서귀포시 장수로 115,제주특별자치도 서귀포시 서홍동 355-1,제주특별자치도서귀포시서홍동,제주특별자치도서귀포시서홍동,"5,013,056,000.0000","5,013,010,600.0000","20,060,701.0000",nan,"5,013,056,000.0000","7,195.5000","3,219.0000",...,639.0000,616.5000,432.0000,242.0000,83.0000,8.5000,0.5000,"3,618.0000","3,618.0000",340.0000,415.5000,365.0000,407.0000,618.5000,546.5000,459.5000,297.5000,145.0000,22.0000,1.5000
417,42110-100311797,춘천우두엘에이치천년나무3단지,0.0000,0.0000,59.4100,59.4100,0.0000,0.0000,0.0000,강원도 춘천시 우두1길 40,강원도 춘천시 우두동 1088,강원도춘천시우두동,강원도춘천시우두동,"4,211,070,500.0000","4,211,011,800.0000","19,981,012.0000",nan,"4,211,070,500.0000","23,027.0000","9,570.0000",...,"1,812.0000","1,793.0000","1,367.0000",694.0000,311.0000,30.0000,0.0000,"11,442.0000","11,442.0000","1,060.0000","1,064.0000","1,235.0000","1,579.0000","1,797.0000","1,711.0000","1,507.0000",846.0000,537.0000,104.0000,2.0000
418,26290-100212522,용호행복주택,0.0000,0.0000,152.2900,0.0000,0.0000,0.0000,0.0000,부산광역시 남구 동명로 183,부산광역시 남구 용호동 376-9,부산광역시남구용호동,부산광역시남구용호동,"2,629,058,500.0000","2,629,010,700.0000","19,950,101.0000",nan,"2,629,058,500.0000","20

In [243]:
mrg_tr.shape

(421, 64)

#### 군포시 도마교동이 원래 대야동이었는데 송부동이되었다.

대야동 정보와 merge하기 

In [244]:
mrg_tr[mrg_tr.행정기관코드_x.isnull()]["juso"]

371    경기도군포시도마교동
Name: juso, dtype: object

In [245]:
mapping1[mapping1.법정동코드 == 4141010900]["주소"]

3562    경기도군포시도마교동
Name: 주소, dtype: object

In [246]:
mapping1.columns

Index(['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드', '동리명', '생성일자', '말소일자',
       '행정기관코드_x', '행정기관_x', '총인구수_x', '세대수', '세대당 인구', '남자 인구수', '여자 인구수',
       '남여 비율', '행정기관코드_y', '행정기관_y', '총인구수_y', '총연령구간인구수', '총0~9세', '총10~19세',
       '총20~29세', '총30~39세', '총40~49세', '총50~59세', '총60~69세', '총70~79세',
       '총80~89세', '총90~99세', '총100세이상', '남 인구수', '남연령구간인구수', '남0~9세',
       '남10~19세', '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세',
       '남70~79세', '남80~89세', '남90~99세', '남100세 이상', '여 인구수', '여연령구간인구수',
       '여0~9세', '여10~19세', '여20~29세', '여30~39세', '여40~49세', '여50~59세',
       '여60~69세', '여70~79세', '여80~89세', '여90~99세', '여100세 이상', '주소'],
      dtype='object')

In [247]:
mrg_tr.loc[mrg_tr.juso =="경기도군포시도마교동",['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드',
       '동리명', '생성일자', '말소일자', '행정기관코드_x', '행정기관_x', '총인구수_x', '세대수_y',
       '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율', '행정기관코드_y', '행정기관_y', '총인구수_y',
       '총연령구간인구수', '총0~9세', '총10~19세', '총20~29세', '총30~39세', '총40~49세',
       '총50~59세', '총60~69세', '총70~79세', '총80~89세', '총90~99세', '총100세이상',
       '남 인구수', '남연령구간인구수', '남0~9세', '남10~19세', '남20~29세', '남30~39세',
       '남40~49세', '남50~59세', '남60~69세', '남70~79세', '남80~89세', '남90~99세',
       '남100세 이상', '여 인구수', '여연령구간인구수', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상', '주소']] = list(mapping1[(mapping1.읍면동명 == "대야동")&(mapping1.시군구명 == "군포시")].reset_index().loc[0][1:])

In [248]:
mrg_tr.loc[mrg_tr.juso =="경기도군포시도마교동","동리명"] = "도마교동"
mrg_tr.loc[mrg_tr.juso =="경기도군포시도마교동",["행정기관_x","주소"]] = "경기도군포시도마교동"
mrg_tr.loc[mrg_tr.juso =="경기도군포시도마교동","행정기관_y"] = "경기도 군포시 도마교동"

In [249]:
mrg_tr.isnull().sum()

관리건축물대장PK     24
건물명           24
대지면적          24
건축면적          24
연면적           24
용적률산정연면적      24
용적률           24
세대수_x         24
가구수           24
도로명주소          0
지번주소           0
juso           0
주소             0
행정동코드          0
법정동코드          0
생성일자           0
말소일자         421
행정기관코드_x       0
총인구수_x         0
세대수_y          0
세대당 인구         0
남자 인구수         0
여자 인구수         0
남여 비율          0
행정기관코드_y       0
총인구수_y         0
총연령구간인구수       0
총0~9세          0
총10~19세        0
총20~29세        0
총30~39세        0
총40~49세        0
총50~59세        0
총60~69세        0
총70~79세        0
총80~89세        0
총90~99세        0
총100세이상        0
남 인구수          0
남연령구간인구수       0
남0~9세          0
남10~19세        0
남20~29세        0
남30~39세        0
남40~49세        0
남50~59세        0
남60~69세        0
남70~79세        0
남80~89세        0
남90~99세        0
남100세 이상       0
여 인구수          0
여연령구간인구수       0
여0~9세          0
여10~19세        0
여20~29세        0
여30~39세        0
여40~49세        0
여50~59세       

## test merge 확인

In [250]:
mrg_tst = bld_tst.merge(mapping2,left_on = "juso",right_on = "주소",how = "left")
mrg_tst

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수_x,가구수,도로명주소,지번주소,juso,주소,행정동코드,법정동코드,생성일자,말소일자,행정기관코드_x,총인구수_x,세대수_y,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
0,41210-100196153,광명역세권 휴먼시아,0.0000,13.3200,13.3200,13.3200,0.0000,0.0000,0.0000,경기도 광명시 서원로 25,경기도 광명시 소하동 1391,경기도광명시소하동,경기도광명시소하동,"4,121,064,500.0000","4,121,010,400.0000","19,880,423.0000",nan,"4,121,064,500.0000","41,315.0000","16,225.5000",...,"3,688.5000","3,292.0000","2,276.5000",933.0000,256.0000,27.5000,1.0000,"20,782.5000","20,782.5000","1,833.5000","2,254.0000","2,560.0000","3,028.0000","3,713.0000","3,227.0000","2,449.5000","1,081.0000",535.0000,99.0000,2.5000
1,41210-100193203,광명역세권 휴먼시아,0.0000,456.8600,"3,773.7700","3,723.4400",0.0000,64.0000,0.0000,경기도 광명시 성채안로 26,경기도 광명시 소하동 1373,경기도광명시소하동,경기도광명시소하동,"4,121,064,500.0000","4,121,010,400.0000","19,880,423.0000",nan,"4,121,064,500.0000","41,315.0000","16,225.5000",...,"3,688.5000","3,292.0000","2,276.5000",933.0000,256.0000,27.5000,1.0000,"20,782.5000","20,782.5000","1,833.5000","2,254.0000","2,560.0000","3,028.0000","3,713.0000","3,227.0000","2,449.5000","1,081.0000",535.0000,99.0000,2.5000
2,26260-100181523,부산안락4 휴먼시아,0.0000,487.0600,"6,035.7800","5,996.3400",0.0000,105.0000,0.0000,부산광역시 동래구 충렬대로447번길 38,부산광역시 동래구 안락동 1256,부산광역시동래구안락동,부산광역시동래구안락동,"2,626,074,500.0000","2,626,010,200.0000","19,950,101.0000",nan,"2,626,074,500.0000","21,096.5000","9,018.0000",...,"1,548.0000","1,709.5000","1,725.5000",901.0000,258.0000,19.0000,1.0000,"10,780.5000","10,780.5000",625.5000,892.0000,"1,105.0000","1,143.0000","1,605.0000","1,905.0000","1,999.0000","1,005.5000",430.5000,64.5000,5.5000
3,45140-100188059,장신휴먼시아3단지,0.0000,503.1300,"6,006.8700","5,953.7800",0.0000,84.0000,0.0000,전라북도 익산시 오산면 선화로4길 14,전라북도 익산시 오산면 장신리 1126,전라북도익산시오산면장신리,전라북도익산시오산면장신리,"4,514,031,000.0000","4,514,031,022.0000","19,950,510.0000",nan,"4,514,031,000.0000","7,783.0000","3,889.0000",...,545.0000,676.0000,733.0000,391.0000,173.0000,19.0000,2.0000,"3,931.0000","3,931.0000",254.0000,304.0000,307.0000,333.0000,478.0000,634.0000,750.0000,479.0000,324.0000,68.0000,0.0000
4,41135-100221777,봇들마을,0.0000,237.0540,361.7890,361.7890,0.0000,0.0000,0.0000,경기도 성남시 분당구 동판교로 212,경기도 성남시 분당구 삼평동 728,경기도성남시분당구삼평동,경기도성남시분당구삼평동,"4,113,563,250.0000","4,113,510,900.0000","20,045,661.5000",nan,"4,113,563,250.0000","19,109.5000","6,799.0000",...,"1,619.5000","1,597.5000","1,010.5000",414.0000,164.0000,18.0000,1.0000,"9,785.5000","9,785.5000",859.0000,"1,175.0000","1,184.5000","1,324.0000","1,821.5000","1,608.5000","1,041.5000",462.0000,247.5000,61.0000,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,50130-100342705,서귀포성산엘에이치아파트,0.0000,40.9200,40.9200,40.9200,0.0000,0.0000,0.0000,제주특별자치도 서귀포시 성산읍 고성동서로 33,제주특별자치도 서귀포시 성산읍 고성리 1141-1,제주특별자치도서귀포시성산읍고성리,제주특별자치도서귀포시성산읍고성리,"5,013,025,900.0000","5,013,025,924.0000","20,060,701.0000",nan,"5,013,025,900.0000","15,478.0000","8,165.0000",...,"1,228.0000","1,743.0000","1,507.0000",732.0000,308.0000,23.0000,0.0000,"7,373.0000","7,373.0000",431.0000,488.0000,649.0000,662.0000,876.0000,"1,385.0000","1,273.0000",805.0000,644.0000,152.0000,8.0000
146,43130-100285480,충주호암1단지,0.0000,211.0500,207.4500,207.4500,0.0000,0.0000,0.0000,충청북도 충주시 호암수청2로 56,충청북도 충주시 호암동 1066,충청북도충주시호암동,충청북도충주시호암동,"4,313,058,000.0000","4,313,010,700.0000","19,880,423.0000",nan,"4,313,058,000.0000","18,468.0000","7,351.0000",...,"1,664.0000","1,414.0000",899.0000,344.0000,117.0000,3.0000,0.0000,"9,306.0000","9,306.0000",996.0000,"1,179.0000",990.0000,"1,314.0000","1,727.0000","1,449.0000",923.0000,417.0000,258.0000,52.0000,1.0000
147,42790-100205071,화천신읍 공공실버주택,0.0000,34.4400,0.0000,0.0000,0.0000,0.0000,0.0000,강원도 화천군 화천읍 산천어길 196,강원도 화천군 화천읍 신읍리 1818,강원도화천군화천읍신읍리,강원도화천군화천읍신읍리,"4,279,025,000.0000","4,279,025,025.0000",

In [251]:
mrg_tst.isnull().sum()

관리건축물대장PK      5
건물명            5
대지면적           5
건축면적           5
연면적            5
용적률산정연면적       5
용적률            5
세대수_x          5
가구수            5
도로명주소          0
지번주소           0
juso           0
주소             0
행정동코드          0
법정동코드          0
생성일자           0
말소일자         150
행정기관코드_x       0
총인구수_x         0
세대수_y          0
세대당 인구         0
남자 인구수         0
여자 인구수         0
남여 비율          0
행정기관코드_y       0
총인구수_y         0
총연령구간인구수       0
총0~9세          0
총10~19세        0
총20~29세        0
총30~39세        0
총40~49세        0
총50~59세        0
총60~69세        0
총70~79세        0
총80~89세        0
총90~99세        0
총100세이상        0
남 인구수          0
남연령구간인구수       0
남0~9세          0
남10~19세        0
남20~29세        0
남30~39세        0
남40~49세        0
남50~59세        0
남60~69세        0
남70~79세        0
남80~89세        0
남90~99세        0
남100세 이상       0
여 인구수          0
여연령구간인구수       0
여0~9세          0
여10~19세        0
여20~29세        0
여30~39세        0
여40~49세        0
여50~59세       

In [252]:
mrg_tst[mrg_tst.행정기관코드_x.isnull()]["juso"]

Series([], Name: juso, dtype: object)

In [253]:
mrg_tst[mrg_tst.행정기관코드_x.isnull()]

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수_x,가구수,도로명주소,지번주소,juso,주소,행정동코드,법정동코드,생성일자,말소일자,행정기관코드_x,총인구수_x,세대수_y,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상


#### 부곡동이 군포2동이었는데 송부동이 되었다. 

In [254]:
pop1[pop1.행정기관코드 == 4141052000]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
1622,4141052000,경기도 군포시 군포2동,53205,20048,2.6500,26356,26849,0.9800


In [255]:
mapping1[mapping1.행정동코드 ==4141052000]

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자,행정기관코드_x,행정기관_x,총인구수_x,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,행정기관코드_y,행정기관_y,총인구수_y,총연령구간인구수,...,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상,주소
3546,4141052000,경기도,군포시,군포2동,4141010100,당동,19890101,nan,"4,141,052,000.0000",경기도군포시군포2동,"53,205.0000","20,048.0000",2.6500,"26,356.0000","26,849.0000",0.9800,"4,141,052,000.0000",경기도 군포시 군포2동,"53,205.0000","53,205.0000",...,"4,422.0000","3,183.0000","1,205.0000",402.0000,43.0000,2.0000,"26,849.0000","26,849.0000","2,193.0000","2,943.0000","3,438.0000","3,428.0000","4,637.0000","4,812.0000","3,200.0000","1,318.0000",742.0000,136.0000,2.0000,경기도군포시당동
3547,4141052000,경기도,군포시,군포2동,4141010300,부곡동,19890101,nan,"4,141,052,000.0000",경기도군포시군포2동,"53,205.0000","20,048.0000",2.6500,"26,356.0000","26,849.0000",0.9800,"4,141,052,000.0000",경기도 군포시 군포2동,"53,205.0000","53,205.0000",...,"4,422.0000","3,183.0000","1,205.0000",402.0000,43.0000,2.0000,"26,849.0000","26,849.0000","2,193.0000","2,943.0000","3,438.0000","3,428.0000","4,637.0000","4,812.0000","3,200.0000","1,318.0000",742.0000,136.0000,2.0000,경기도군포시부곡동


In [256]:
mrg_tst.loc[mrg_tst.juso =="경기도군포시부곡동",['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드',
       '동리명', '생성일자', '말소일자', '행정기관코드_x', '행정기관_x', '총인구수_x', '세대수_y',
       '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율', '행정기관코드_y', '행정기관_y', '총인구수_y',
       '총연령구간인구수', '총0~9세', '총10~19세', '총20~29세', '총30~39세', '총40~49세',
       '총50~59세', '총60~69세', '총70~79세', '총80~89세', '총90~99세', '총100세이상',
       '남 인구수', '남연령구간인구수', '남0~9세', '남10~19세', '남20~29세', '남30~39세',
       '남40~49세', '남50~59세', '남60~69세', '남70~79세', '남80~89세', '남90~99세',
       '남100세 이상', '여 인구수', '여연령구간인구수', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상', '주소']] = list(mapping1[(mapping1.읍면동명 == "군포2동")&(mapping1.시군구명 == "군포시")].reset_index().loc[0][1:])

In [257]:
mrg_tst.loc[mrg_tst.juso =="경기도군포시부곡동","동리명"] = "부곡동"
mrg_tst.loc[mrg_tst.juso =="경기도군포시부곡동",["행정기관_x","주소"]] = "경기도군포시부곡동"
mrg_tst.loc[mrg_tst.juso =="경기도군포시부곡동","행정기관_y"] = "경기도 군포시 부곡동"

In [258]:
mrg_tr.columns

Index(['관리건축물대장PK', '건물명', '대지면적', '건축면적', '연면적', '용적률산정연면적', '용적률', '세대수_x',
       '가구수', '도로명주소', '지번주소', 'juso', '주소', '행정동코드', '법정동코드', '생성일자', '말소일자',
       '행정기관코드_x', '총인구수_x', '세대수_y', '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율',
       '행정기관코드_y', '총인구수_y', '총연령구간인구수', '총0~9세', '총10~19세', '총20~29세',
       '총30~39세', '총40~49세', '총50~59세', '총60~69세', '총70~79세', '총80~89세',
       '총90~99세', '총100세이상', '남 인구수', '남연령구간인구수', '남0~9세', '남10~19세',
       '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세', '남70~79세',
       '남80~89세', '남90~99세', '남100세 이상', '여 인구수', '여연령구간인구수', '여0~9세',
       '여10~19세', '여20~29세', '여30~39세', '여40~49세', '여50~59세', '여60~69세',
       '여70~79세', '여80~89세', '여90~99세', '여100세 이상', '시도명', '시군구명', '읍면동명',
       '동리명', '행정기관_x', '행정기관_y'],
      dtype='object')

In [259]:
mrg_tr = mrg_tr[['도로명주소', '총인구수_x', 
       '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율', '총0~9세', '총10~19세', '총20~29세', '총30~39세', '총40~49세',
       '총50~59세', '총60~69세', '총70~79세', '총80~89세', '총90~99세', '총100세이상',
'남0~9세', '남10~19세', '남20~29세', '남30~39세',
       '남40~49세', '남50~59세', '남60~69세', '남70~79세', '남80~89세', '남90~99세',
       '남100세 이상',  '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상']]
mrg_tst = mrg_tst[['도로명주소', '총인구수_x', 
       '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율',  '총0~9세', '총10~19세', '총20~29세', '총30~39세', '총40~49세',
       '총50~59세', '총60~69세', '총70~79세', '총80~89세', '총90~99세', '총100세이상',
'남0~9세', '남10~19세', '남20~29세', '남30~39세',
       '남40~49세', '남50~59세', '남60~69세', '남70~79세', '남80~89세', '남90~99세',
       '남100세 이상', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상']]

In [260]:
mrg_tr.columns 

Index(['도로명주소', '총인구수_x', '세대당 인구', '남자 인구수', '여자 인구수', '남여 비율', '총0~9세',
       '총10~19세', '총20~29세', '총30~39세', '총40~49세', '총50~59세', '총60~69세',
       '총70~79세', '총80~89세', '총90~99세', '총100세이상', '남0~9세', '남10~19세',
       '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세', '남70~79세',
       '남80~89세', '남90~99세', '남100세 이상', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상'],
      dtype='object')

In [261]:
mrg_tr.columns = ['도로명주소', '총인구수', '세대당_인구', '남자_인구수', '여자_인구수', '남/여비율', '총0~9세',
       '총10~19세', '총20~29세', '총30~39세', '총40~49세', '총50~59세', '총60~69세',
       '총70~79세', '총80~89세', '총90~99세', '총100세이상', '남0~9세', '남10~19세',
       '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세', '남70~79세',
       '남80~89세', '남90~99세', '남100세 이상', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상']
mrg_tst.columns = mrg_tr.columns

In [262]:
mrg_tr["남0~19세"] = mrg_tr['남0~9세']+mrg_tr['남10~19세']
mrg_tr["여0~19세"] = mrg_tr['여0~9세']+mrg_tr['여10~19세']
mrg_tr["총0~19세"] = mrg_tr['총0~9세']+mrg_tr['총10~19세']
mrg_tr["남20~39세"] = mrg_tr['남20~29세']+mrg_tr['남30~39세']
mrg_tr["여20~39세"] = mrg_tr['여20~29세']+mrg_tr['여30~39세']
mrg_tr["총20~39세"] = mrg_tr['남20~39세']+mrg_tr['여20~39세']
mrg_tr["남40~69세"] = mrg_tr['남40~49세']+mrg_tr['남50~59세']+mrg_tr['남60~69세']
mrg_tr["여40~69세"] = mrg_tr['여40~49세']+mrg_tr['여50~59세']+mrg_tr['여60~69세']
mrg_tr["총40~69세"] = mrg_tr['남40~69세']+mrg_tr['여40~69세']
mrg_tr["남70세이상"] = mrg_tr['남70~79세']+mrg_tr['남80~89세']+mrg_tr['남90~99세']+mrg_tr["남100세 이상"]
mrg_tr["여70세이상"] = mrg_tr['여70~79세']+mrg_tr['여80~89세']+mrg_tr['여90~99세']+mrg_tr["여100세 이상"]
mrg_tr["총70세이상"] = mrg_tr["여70세이상"]+mrg_tr["남70세이상"]
mrg_tr["남/여_0~19세"] = mrg_tr['남0~19세']/mrg_tr['여10~19세']
mrg_tr["남/여_20~39세"] = mrg_tr['남20~39세']/mrg_tr['여20~39세']
mrg_tr["남/여_40~69세"] = mrg_tr['남40~69세']/mrg_tr['여40~69세']
mrg_tr["남/여_70세이상"] = mrg_tr['남70세이상']/mrg_tr['여70세이상']

mrg_tr["0~19세_비율"] = mrg_tr['총0~19세']/mrg_tr["총인구수"]
mrg_tr["20~39세_비율"] = mrg_tr['총20~39세']/mrg_tr["총인구수"]
mrg_tr["40~69세_비율"] = mrg_tr['총40~69세']/mrg_tr["총인구수"]
mrg_tr["70세이상_비율"] = mrg_tr['총70세이상']/mrg_tr["총인구수"]

In [263]:
mrg_tst["남0~19세"] = mrg_tst['남0~9세']+mrg_tst['남10~19세']
mrg_tst["여0~19세"] = mrg_tst['여0~9세']+mrg_tst['여10~19세']
mrg_tst["총0~19세"] = mrg_tst['총0~9세']+mrg_tst['총10~19세']
mrg_tst["남20~39세"] = mrg_tst['남20~29세']+mrg_tst['남30~39세']
mrg_tst["여20~39세"] = mrg_tst['여20~29세']+mrg_tst['여30~39세']
mrg_tst["총20~39세"] = mrg_tst['남20~39세']+mrg_tst['여20~39세']
mrg_tst["남40~69세"] = mrg_tst['남40~49세']+mrg_tst['남50~59세']+mrg_tst['남60~69세']
mrg_tst["여40~69세"] = mrg_tst['여40~49세']+mrg_tst['여50~59세']+mrg_tst['여60~69세']
mrg_tst["총40~69세"] = mrg_tst['남40~69세']+mrg_tst['여40~69세']
mrg_tst["남70세이상"] = mrg_tst['남70~79세']+mrg_tst['남80~89세']+mrg_tst['남90~99세']+mrg_tst["남100세 이상"]
mrg_tst["여70세이상"] = mrg_tst['여70~79세']+mrg_tst['여80~89세']+mrg_tst['여90~99세']+mrg_tst["여100세 이상"]
mrg_tst["총70세이상"] = mrg_tst["여70세이상"]+mrg_tst["남70세이상"]
mrg_tst["남/여_0~19세"] = mrg_tst['남0~19세']/mrg_tst['여10~19세']
mrg_tst["남/여_20~39세"] = mrg_tst['남20~39세']/mrg_tst['여20~39세']
mrg_tst["남/여_40~69세"] = mrg_tst['남40~69세']/mrg_tst['여40~69세']
mrg_tst["남/여_70세이상"] = mrg_tst['남70세이상']/mrg_tst['여70세이상']

mrg_tst["0~19세_비율"] = mrg_tst['총0~19세']/mrg_tst["총인구수"]
mrg_tst["20~39세_비율"] = mrg_tst['총20~39세']/mrg_tst["총인구수"]
mrg_tst["40~69세_비율"] = mrg_tst['총40~69세']/mrg_tst["총인구수"]
mrg_tst["70세이상_비율"] = mrg_tst['총70세이상']/mrg_tst["총인구수"]

In [264]:
mrg_tr.columns

Index(['도로명주소', '총인구수', '세대당_인구', '남자_인구수', '여자_인구수', '남/여비율', '총0~9세',
       '총10~19세', '총20~29세', '총30~39세', '총40~49세', '총50~59세', '총60~69세',
       '총70~79세', '총80~89세', '총90~99세', '총100세이상', '남0~9세', '남10~19세',
       '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세', '남70~79세',
       '남80~89세', '남90~99세', '남100세 이상', '여0~9세', '여10~19세', '여20~29세',
       '여30~39세', '여40~49세', '여50~59세', '여60~69세', '여70~79세', '여80~89세',
       '여90~99세', '여100세 이상', '남0~19세', '여0~19세', '총0~19세', '남20~39세',
       '여20~39세', '총20~39세', '남40~69세', '여40~69세', '총40~69세', '남70세이상',
       '여70세이상', '총70세이상', '남/여_0~19세', '남/여_20~39세', '남/여_40~69세',
       '남/여_70세이상', '0~19세_비율', '20~39세_비율', '40~69세_비율', '70세이상_비율'],
      dtype='object')

In [268]:
mrg_tr = mrg_tr[['도로명주소','총인구수', '세대당_인구',  '남/여비율','남/여_0~19세', '남/여_20~39세', '남/여_40~69세',
       '남/여_70세이상', '0~19세_비율', '20~39세_비율', '40~69세_비율', '70세이상_비율']]
mrg_tst = mrg_tst[['도로명주소','총인구수', '세대당_인구',  '남/여비율','남/여_0~19세', '남/여_20~39세', '남/여_40~69세',
       '남/여_70세이상', '0~19세_비율', '20~39세_비율', '40~69세_비율', '70세이상_비율']]

In [269]:
mrg_tr.to_csv("ProcessedData/pop_tr.csv",index=False)
mrg_tst.to_csv("ProcessedData/pop_tst.csv",index=False)

In [60]:
pop_tr.shape

NameError: name 'pop_tr' is not defined

mrg1.to_csv("/Users/seungji/Desktop/Dacon/mrg1.csv",index = False)